In [12]:
import pandas as pd

Com o arquivo `dicionario_tratado.csv` em mãos (gerado no etl_dicionario), vamos prosseguir com a leitura do .txt.
Para iniciar, o pandas fornece uma função especifica para estes modelos, e vamos usar o nosso csv para tal.

In [13]:
dic = pd.read_csv("dicionario_tratado.csv")
print(dic.head())

  Posição inicial  Tamanho Código da variável Quesito     Descrição quesito  \
0               1      2.0              V0001     NaN  Unidade da Federação   
1               1      2.0              V0001     NaN  Unidade da Federação   
2               1      2.0              V0001     NaN  Unidade da Federação   
3               1      2.0              V0001     NaN  Unidade da Federação   
4               1      2.0              V0001     NaN  Unidade da Federação   

  Tipo categoria Descrição categoria  
0             11            Rondônia  
1             12                Acre  
2             13            Amazonas  
3             14             Roraima  
4             15                Pará  


Vamos começar lidando com as linhas com mesmo código da variável, mas com valores de variáveis diferentes

In [ ]:
dic_vars = (
    dic.groupby('Código da variável', as_index=False)
       .agg({'Posição inicial':'first', 'Tamanho':'first'})
       .rename(columns={'Código da variável':'var'})
)
# garantir inteiros
dic_vars['Posição inicial'] = dic_vars['Posição inicial'].astype('Int64')
dic_vars['Tamanho'] = dic_vars['Tamanho'].astype('Int64')

# Posições
dic_vars['start0'] = dic_vars['Posição inicial'] - 1
dic_vars['end0']   = dic_vars['start0'] + dic_vars['Tamanho']

colspecs = list(zip(dic_vars['start0'].astype(int), dic_vars['end0'].astype(int)))
names = dic_vars['var'].tolist()

print(names[0])
print(colspecs[0])



A001
(34, 35)
       var  Posição inicial  Tamanho  start0  end0
983  V0001                1        2       0     2


Agora, vamos criar o mapping para as variáveis

In [18]:
dic['Tipo categoria'] = dic['Tipo categoria'].astype('string')
dic['Descrição categoria'] = dic['Descrição categoria'].astype('string')

code_maps = {}
for var, g in dic.groupby('Código da variável'):
    # só linhas que realmente têm categoria
    g = g.dropna(subset=['Tipo categoria', 'Descrição categoria'])
    if not g.empty:
        code_maps[var] = dict(zip(g['Tipo categoria'], g['Descrição categoria']))

print(code_maps['V0001'])


{'11': 'Rondônia', '12': 'Acre', '13': 'Amazonas', '14': 'Roraima', '15': 'Pará', '16': 'Amapá', '17': 'Tocantins', '21': 'Maranhão', '22': 'Piauí', '23': 'Ceará', '24': 'Rio Grande do Norte', '25': 'Paraíba', '26': 'Pernambuco', '27': 'Alagoas', '28': 'Sergipe', '29': 'Bahia', '31': 'Minas Gerais', '32': 'Espírito Santo', '33': 'Rio de Janeiro', '35': 'São Paulo', '41': 'Paraná', '42': 'Santa Catarina', '43': 'Rio Grande do Sul', '50': 'Mato Grosso do Sul', '51': 'Mato Grosso', '52': 'Goiás', '53': 'Distrito Federal'}


Finalmente, podemos seguir com a leitura dos microdados.

In [ ]:
df = pd.read_fwf(
    'dados.txt',
    colspecs=colspecs,
    names=names,
    dtype=str,
    encoding='utf-8'
)
# Mapping das variaveis
for var, mapping in code_maps.items():
    if var in df.columns:
        df[var] = df[var].astype('string').map(mapping).fillna(df[var])